In [1]:
from dotenv import load_dotenv
from os.path import join, dirname
from google.cloud import bigquery

In [2]:
dotenv_path = '.envtemp'
load_dotenv(dotenv_path)

True

In [3]:
import json
srcJsonPath = '/home/pardhu448/Projects/indusAction/redesignBocwGrievanceSystem/BoCWDataPipeline/BoCWDataPipeline/data/dailyMongoDump/snapShot.json'

In [4]:
with open(srcJsonPath, 'r') as j:
     contents = json.loads(j.read())

In [5]:
def replaceAllDollarsInKeys(d):
    new = {}
    for k, v in d.items():
        k = k.encode('raw-unicode-escape').split(b'\\u')[0].decode()
        if isinstance(v, dict):
            v = replaceAllDollarsInKeys(v)
        new[k.replace('$', '_')] = v
    return new
parsedJsonArray = [replaceAllDollarsInKeys(eaDoc) for eaDoc in contents][:20]

AttributeError: 'dict' object has no attribute 'encode'

In [70]:
srcJsonPath2 = '/home/pardhu448/Projects/indusAction/redesignBocwGrievanceSystem/BoCWDataPipeline/BoCWDataPipeline/data/dailyMongoDump/snapShot4.json'
with open(srcJsonPath2, 'w') as f:
    f.write('\n'.join(map(json.dumps, parsedJsonArray)))

In [71]:
BQ_DATASET = 'BoCWGrievancesData'
client = bigquery.Client()
dataset_id = BQ_DATASET
table_id = 'BoCWMasterTable_Ver00'
  
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
job_config.autodetect = True
    
#with open(srcJsonPath2, "rb") as source_file:
#      job = client.load_table_from_file(source_file, 
#              table_ref, 
#              location="asia-south1",  # Must match the destination dataset location.
#              job_config=job_config)  # API request
  
#job.result()  # Waits for table load to complete.

LoadJob<project=bocwdatapipeline, location=asia-south1, id=994bf6ed-3a05-417e-b78b-ba4ec3be48b8>